In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas.tseries import offsets
import random as rand
import matplotlib.pyplot as plot


# -----------------------
# Interview Q: 05/31/2023
    # Create a retention chart showing the new users and the percent of users still active

url = 'https://raw.githubusercontent.com/erood/interviewqs.com_code_snippets/master/Datasets/online_retail.csv'

data_0531 = pd.read_csv(url)

In [73]:
table = (data_0531
    .assign(
        real_date = lambda dfx: pd.to_datetime(dfx["InvoiceDate"]).dt.date, 
        month_end = lambda dfy: dfy["real_date"] + offsets.MonthEnd(), 
        signup_month = lambda dfz: (dfz.groupby("CustomerID")
            ["month_end"]
            .transform("min")), 
        periods_af = lambda dfa: (dfa["month_end"].dt.to_period("M").view(dtype='int64') - dfa["signup_month"].dt.to_period("M").view(dtype='int64'))
    )
    .groupby(["signup_month", "periods_af"])
    .agg(
        users = ("CustomerID", "nunique")
    )
    .reset_index()
    .assign(
        new_users = lambda dfb: (dfb.groupby("signup_month")
            ["users"]
            .transform("max"))
    )
    .query("periods_af > 0")
    .assign(
        retention = lambda dfc: dfc["users"] / dfc["new_users"], 
        signup_month = lambda dfd: dfd["signup_month"]#.dt.strftime('%b %d,%Y')
    )
    .pivot_table(
        values = "retention", 
        index = ["signup_month", "new_users"], 
        columns = "periods_af", 
    )
    .reset_index()
    .rename_axis(None, axis=1)
)

(table
    .style
    .background_gradient(subset = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], axis = 1)
    .applymap(lambda x: 'background-color: transparent; color: transparent;' if pd.isnull(x) else '')
    .format({
        "signup_month": "{:%b %d, %Y}", 
        1: "{:,.0%}", 2: "{:,.0%}", 3: "{:,.0%}", 4: "{:,.0%}", 5: "{:,.0%}", 6: "{:,.0%}", 
        7: "{:,.0%}", 8: "{:,.0%}", 9: "{:,.0%}", 10: "{:,.0%}", 11: "{:,.0%}", 12: "{:,.0%}"
    })
)


,signup_month,new_users,1,2,3,4,5,6,7,8,9,10,11,12
0,"Dec 31, 2010",948,37%,35%,38%,37%,39%,38%,36%,35%,40%,37%,50%,30%
1,"Jan 31, 2011",403,24%,28%,26%,33%,29%,26%,25%,31%,34%,36%,17%,nan%
2,"Feb 28, 2011",374,24%,20%,26%,27%,26%,27%,28%,27%,30%,11%,nan%,nan%
3,"Mar 31, 2011",446,20%,26%,21%,22%,18%,26%,25%,29%,11%,nan%,nan%,nan%
4,"Apr 30, 2011",317,22%,22%,22%,20%,25%,22%,26%,9%,nan%,nan%,nan%,nan%
5,"May 31, 2011",270,24%,19%,17%,21%,26%,25%,12%,nan%,nan%,nan%,nan%,nan%
6,"Jun 30, 2011",238,19%,19%,26%,24%,34%,12%,nan%,nan%,nan%,nan%,nan%,nan%
7,"Jul 31, 2011",193,20%,21%,23%,27%,11%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
8,"Aug 31, 2011",166,26%,23%,26%,14%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
9,"Sep 30, 2011",291,30%,32%,14%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


In [50]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

# Reset the index without creating a new "name" column
df_reset = df.reset_index(drop=True)

print(df_reset)


   A  B
0  1  4
1  2  5
2  3  6
